In [11]:
import numpy as np

---
# Generic class definitions for various operators/nodes

In [12]:
class Node(object):
    """
    Base object for all inputs and outputs.
    """
    def __init__(self, value, grad):
        self.value = value
        self.gradient = grad

In [13]:
class MultiplyNode(object):
    """
    Multiplies two inputs
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value * self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = self.x2.value * self.output.gradient
        self.x2.gradient = self.x1.value * self.output.gradient

In [14]:
class AddNode(object):    
    """
    Adds two inputs x1 and x2.
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value + self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = 1 * self.output.gradient
        self.x2.gradient = 1 * self.output.gradient

In [15]:
class SigmoidNode(object):
    """
    Adds a sigmoid non-linearity to a single input
    """
    def forward(self, x):
        self.x = x
        self.output = Node(1/(1 + np.exp(-1 * self.x.value)), 0.0)
        return self.output
        
    def backward(self):
        s = 1/(1 + np.exp(-1 * self.x.value))
        self.x.gradient = (s * (1 - s)) * self.output.gradient

In [29]:
class Perceptron(object):
    def __init__(self, x1, x2, alpha=0.001):
        ### Hyper parameters
        self.alpha = alpha
        ### Initializing weights/bias to a random float between -1 and 1.
        self.w1 = Node(np.random.uniform(-1, 1), 0.0)
        self.w2 = Node(np.random.uniform(-1, 1), 0.0)
        self.w3 = Node(np.random.uniform(-1,1),0.0)
        self.w4 = Node(np.random.uniform(-1,1),0.0)
        self.b = Node(np.random.uniform(-1, 1), 0.0)
        ### Input and Output variables
        self.x1 = Node(x1, 0.0)
        self.x2 = Node(x2, 0.0)
        ### Initialize operators nodes required 
        ### for processing the inputs within a perceptron
        self.initialize_operators()
    
    def initialize_operators(self):
        self.w1_mul_x1 = MultiplyNode()
        self.w2_mul_x2 = MultiplyNode()
        self.w1x1_add_w2x2 = AddNode()
        self.w1x1_w2x2_add_b = AddNode()
        self.sigmoid = SigmoidNode()
        self.sigmoid1 = SigmoidNode()
        self.w3_mul_oih= MultiplyNode()
        self.w4_mul_dummy = MultiplyNode()
        self.w3_mul_ioh_add_w4_mul_dummy = AddNode()
        self.w3_mul_ioh_add_w4_mul_dummy_b = AddNode()
    
    def forward(self):
        w1x1 = self.w1_mul_x1.forward(self.w1, self.x1)
        w2x2 = self.w2_mul_x2.forward(self.w2, self.x2)
        w1x1_w2x2 = self.w1x1_add_w2x2.forward(w1x1, w2x2)
        w1x1_w2x2_b = self.w1x1_w2x2_add_b.forward(w1x1_w2x2, self.b)
        sigmoidlayer1=self.sigmoid.forward(w1x1_w2x2_b)
        w3out1=self.w3_mul_oih.forward(self.w3,sigmoidlayer1)
        w2out2=self.w4_mul_dummy.forward(self.w4,1)
        w2out2_w3out3 = self.w3_mul_ioh_add_w4_mul_dummy.forward(w3out1,w2out2)
        w2out2_w3out3_b=self.w3_mul_ioh_add_w4_mul_dummy_b.forward(w2out2_w3out3,self.b)
        self.sigmoid1.forward(w3_mul_ioh_add_w4_mul_dummy_b)
        
        
    def backward(self):
        self.sigmoid1.backward()
        w3_mul_ioh_add_w4_mul_dummy_b.backward()
        w3_mul_ioh_add_w4_mul_dummy.backward()
        w4_mul_dummy.backward()
        w3_mul_oih.backward()
        self.sigmoid.backward()
        self.w1x1_w2x2_add_b.backward()
        self.w1x1_add_w2x2.backward()
        self.w2_mul_x2.backward()
        self.w1_mul_x1.backward()
    
    def update(self):
        self.w4.value -=self.aplha * self.w4.gradient
        self.w3.value -=self.aplha * self.w3.gradient
        self.w1.value -= self.alpha * self.w1.gradient
        self.w2.value -= self.alpha * self.w2.gradient
        self.b.value -= self.alpha * self.b.gradient

### Validation

In [30]:
p = Perceptron(-1,1.0, alpha=0.1)
# number of iterations
N = 100000
# expected output 
target = 0.3481972639817

for i in range(N):
    # Step 1. Forward Pass
    p.forward()
    # Step 2. Calculate Loss. -2 * (y - output) is the gradient of output w.r.t 
    # square loss function.
    p.sigmoid.output.gradient = -2 * (target - p.sigmoid.output.value)
    # Step 3. Backward Pass
    p.backward()
    # Step 4. Update Weights and Bias
    p.update()

p.sigmoid.output.value

AttributeError: 'int' object has no attribute 'value'

---

# Assignment 3

```
X1 ------\
          \
          (P) ---- (P) ---- (T)
          /
X2 ------/
         
```

1. Create a two layer neural network with one perceptron in each layer (see Diagram above). Write a validation code that does along with your implementation. The goal of network is to optimize the two perceptrons to produce the output target `T` given the inputs `X1` and `X2`. Assume the output `O` of each perceptron is

$$ O = \sigma{(w1*x1 + w2*x2 + b)} $$ where

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

Feel free to change the loss function if you like.

